In [ ]:
# Install and import packages

# %conda install pandas numpy emnist matplotlib

# Import all the things
import pandas as pd
import numpy as np
import emnist
import matplotlib.pyplot as plt

In [ ]:
# Load the data, and reshape it
size = 28 # The size of the images

# Extract the training split as images and labels
image, label = emnist.extract_training_samples('byclass')

# Add columns for each pixel value (28x28 = 784 columns)
raw_train = pd.DataFrame( image.reshape(-1, size*size) )

# Add a column showing which split the data is from
raw_train['split'] = 'train'

# Add a column showing the label
raw_train['label'] = label

# Add a column with the image data as a 28x28 array
raw_train['image'] = list(image)


# Repeat for the test split
image, label = emnist.extract_test_samples('byclass')
raw_test = pd.DataFrame( image.reshape(-1, size*size) )
raw_test['label'] = label
raw_test['split'] = 'test'
raw_test['image'] = list(image)

In [ ]:
# We can get the image back from the dataframe by reshaping it
# into a NumPy array (which is what the image was originally)

# Creat a NumPy array from the columns of pixels
image_np = np.array(raw_train.iloc[:,0:784])

# Reshape it into a 28x28 image
image_np = np.array(image_np.reshape(-1, 28, 28))

# And plot the first one, image_np[0], with matplotlib's pyplot
plt.imshow(image_np[0], cmap='gray')

In [ ]:
# We can also get the image back from the 'image' column of the dataframe,
# which is a list of NumPy arrays
plt.imshow(raw_train['image'][0], cmap='gray')

In [ ]:
# Now let's mess up the data a bit

# Copy the splits into new dataframes to mess up
dirty_train = raw_train.copy()
dirty_test  = raw_test.copy()

# Add a column for previous prediction score
dirty_train['predict'] = np.random.rand(dirty_train.shape[0])
dirty_test['predict']  = np.random.rand(dirty_test.shape[0])